##Блок 1. Базовые сущности
🔹 Объяснение

BoundedStat – дескриптор, который контролирует значения характеристик (HP, MP, сила и т.д.).

Например, HP не может быть меньше 0 и больше 9999.

Каждый раз, когда мы присваиваем новое значение, оно автоматически проверяется.

Human – базовый класс для всех существ в игре.

Хранит общие характеристики: имя, уровень, HP, MP, силу, ловкость и интеллект.

Свойство is_alive проверяет, жив ли персонаж.

Character – абстрактный класс для всех игровых персонажей (героев и боссов).

Определяет обязательные методы basic_attack и use_skill, которые должны реализовывать все подклассы.

In [24]:
import random
from abc import ABC, abstractmethod

# ==================== Блок 1. Базовые сущности ====================

class BoundedStat:
    def __init__(self, name, min_value=0, max_value=9999):
        self.name = "_" + name
        self.min_value = min_value
        self.max_value = max_value

    def __get__(self, instance, owner):
        return getattr(instance, self.name)

    def __set__(self, instance, value):
        if value < self.min_value:
            value = self.min_value
        elif value > self.max_value:
            value = self.max_value
        setattr(instance, self.name, value)

class Human:
    hp = BoundedStat("hp", 0)
    mp = BoundedStat("mp", 0)
    strength = BoundedStat("strength", 1)
    agility = BoundedStat("agility", 1)
    intellect = BoundedStat("intellect", 1)

    def __init__(self, name, level=1, hp=100, mp=50, strength=10, agility=10, intellect=10):
        self.name = name
        self.level = level
        self.hp = hp
        self.mp = mp
        self.strength = strength
        self.agility = agility
        self.intellect = intellect

    @property
    def is_alive(self):
        return self.hp > 0

    def __str__(self):
        return f"{self.name} (Lvl {self.level}) HP:{self.hp} MP:{self.mp}"

class Character(Human, ABC):
    @abstractmethod
    def basic_attack(self, target):
        pass

    @abstractmethod
    def use_skill(self, target):
        pass



##Блок 2. Персонажи
🔹 Объяснение

Human — базовый класс для всех существ (герои и босс).

Содержит общие характеристики: HP, MP, силу, ловкость, интеллект, уровень и имя.

Свойство is_alive проверяет, жив ли персонаж.

Метод __str__ выводит информацию о персонаже.

In [25]:
# ==================== Блок 2. Персонажи ====================
class Tank(Character):
    def __init__(self, name, hp, mp, strength, agility, intellect, defense, level=1):
        super().__init__(name, level, hp, mp, strength, agility, intellect)
        self.defense = defense

    def basic_attack(self, target):
        damage = self.strength
        target.hp -= damage
        print(f"{self.name} бьёт {target.name} на {damage} урона!")

    def use_skill(self, target=None):
        shield_value = self.defense * 2
        print(f"{self.name} поднимает щит 🛡️, снижая урон на {shield_value}!")

class Warrior(Character):
    def __init__(self, name, hp, mp, strength, agility, intellect, level=1):
        super().__init__(name, level, hp, mp, strength, agility, intellect)

    def basic_attack(self, target):
        damage = self.strength * 2
        target.hp -= damage
        print(f"{self.name} атакует {target.name} на {damage} урона!")

    def use_skill(self, target):
        if self.mp >= 10:
            damage = self.strength * 4
            target.hp -= damage
            self.mp -= 10
            print(f"{self.name} использует 'Мощный удар' и наносит {damage} урона {target.name}!")
        else:
            self.basic_attack(target)

class Mage(Character):
    def __init__(self, name, hp, mp, strength, agility, intellect, level=1):
        super().__init__(name, level, hp, mp, strength, agility, intellect)

    def basic_attack(self, target):
        damage = self.intellect * 2
        target.hp -= damage
        print(f"{self.name} наносит {damage} магического урона {target.name}!")

    def use_skill(self, target):
        if self.mp >= 15:
            damage = self.intellect * 5
            target.hp -= damage
            self.mp -= 15
            print(f"{self.name} кидает 'Огненный шар' и наносит {damage} урона {target.name}!")
        else:
            self.basic_attack(target)

class Healer(Character):
    def __init__(self, name, hp, mp, strength, agility, intellect, level=1):
        super().__init__(name, level, hp, mp, strength, agility, intellect)

    def basic_attack(self, target):
        damage = self.intellect
        target.hp -= damage
        print(f"{self.name} наносит {damage} урона посохом {target.name}!")

    def use_skill(self, target):
        if self.mp >= 12:
            heal = self.intellect * 4
            target.hp += heal
            self.mp -= 12
            print(f"{self.name} лечит {target.name} на {heal} HP!")
        else:
            self.basic_attack(target)

class Assassin(Character):
    def __init__(self, name, hp, mp, strength, agility, intellect, crit_chance=0.3, level=1):
        super().__init__(name, level, hp, mp, strength, agility, intellect)
        self.crit_chance = crit_chance

    def basic_attack(self, target):
        if random.random() < self.crit_chance:
            damage = self.strength * 3
            target.hp -= damage
            print(f"{self.name} наносит критический удар ⚡ на {damage} урона {target.name}!")
        else:
            damage = int(self.strength * 1.5)
            target.hp -= damage
            print(f"{self.name} атакует {target.name} на {damage} урона.")

    def use_skill(self, target):
        print(f"{self.name} использует 'Теневой удар' на {target.name}!")
        self.basic_attack(target)

##Блок 3. Боевая система

BattleLogger — контекстный менеджер для логирования боя. Он выводит ход боя и итоговую информацию в конце.

Battle — основной игровой цикл:

Игрок выбирает какой герой ходит и какое действие выполняет (атака или навык).

Босс автоматически атакует случайного живого героя.

Цикл продолжается, пока все герои или босс не погибнут.

Система проверяет конец боя и выводит результат (победа/поражение).

In [26]:
# ==================== Блок 3. Боевая система ====================
class BattleLogger:
    def __enter__(self):
        self.log = []
        print("⚔️ Начало боя!")
        return self

    def write(self, message):
        self.log.append(message)
        print(message)

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("🏆 Бой окончен!")
        print("\n=== Лог боя ===")
        for entry in self.log:
            print(entry)

class Battle:
    def __init__(self, party, boss):
        self.party = party
        self.boss = boss

    def is_over(self):
        return not any(hero.is_alive for hero in self.party) or not self.boss.is_alive

    def run(self):
        with BattleLogger() as logger:
            while not self.is_over():
                # -------------------- Ход игрока --------------------
                print("\nХод Пати:")
                alive_heroes = [h for h in self.party if h.is_alive]
                for idx, hero in enumerate(alive_heroes):
                    print(f"{idx + 1}. {hero}")

                # Выбираем героя
                while True:
                    try:
                        choice = int(input(f"Выберите героя (1-{len(alive_heroes)}): "))
                        if 1 <= choice <= len(alive_heroes):
                            current = alive_heroes[choice - 1]
                            break
                    except:
                        pass
                    print("Неверный выбор, попробуйте снова.")

                # Выбираем действие
                print(f"\nВыберите действие для {current.name}:")
                print("1 - Basic Attack")
                print("2 - Use Skill")
                while True:
                    action = input("Ваш выбор: ")
                    if action == "1":
                        current.basic_attack(self.boss)
                        break
                    elif action == "2":
                        current.use_skill(self.boss)
                        break
                    else:
                        print("Неверный выбор, попробуйте снова.")

                logger.write(f"{current.name} сделал ход.")

                # -------------------- Ход босса --------------------
                if self.boss.is_alive:
                    print(f"\nХод босса {self.boss.name}:")
                    target = random.choice([h for h in self.party if h.is_alive])
                    self.boss.basic_attack(target)
                    logger.write(f"{self.boss.name} ходит.")

            # -------------------- Конец боя --------------------
            if self.boss.is_alive:
                logger.write("❌ Пати проиграла! Босс победил!")
            else:
                logger.write("✅ Босс повержен!")

##Блок 4. Запуск игры

Позволяет игроку:

Ввести имена своих героев.

Выбрать сложность игры, которая влияет на характеристики босса.

Создаются экземпляры всех героев и босса.

Инициализируется объект Battle и запускается метод run().

Игрок получает интерактивное управление: на каждом ходу можно выбрать, какой герой действует и каким способом.

In [27]:
class Battle:
    def __init__(self, party, boss):
        self.party = party
        self.boss = boss
        self.effects = []

    def is_over(self):
        return not any(hero.is_alive for hero in self.party) or not self.boss.is_alive

    def run(self):
        with BattleLogger() as logger:
            while not self.is_over():
                # -------------------- Ход игрока --------------------
                print("\nХод Пати:")
                alive_heroes = [h for h in self.party if h.is_alive]
                for idx, hero in enumerate(alive_heroes):
                    print(f"{idx + 1}. {hero}")

                # Выбираем героя
                while True:
                    try:
                        choice = int(input(f"Выберите героя (1-{len(alive_heroes)}): "))
                        if 1 <= choice <= len(alive_heroes):
                            current = alive_heroes[choice - 1]
                            break
                    except:
                        pass
                    print("Неверный выбор, попробуйте снова.")

                # Выбираем действие
                print(f"\nВыберите действие для {current.name}:")
                print("1 - Basic Attack")
                print("2 - Use Skill")
                while True:
                    action = input("Ваш выбор: ")
                    if action == "1":
                        current.basic_attack(self.boss)
                        break
                    elif action == "2":
                        current.use_skill(self.boss)
                        break
                    else:
                        print("Неверный выбор, попробуйте снова.")

                logger.write(f"{current.name} сделал ход.")

                # -------------------- Ход босса --------------------
                if self.boss.is_alive:
                    print(f"\nХод босса {self.boss.name}:")
                    # Для простоты босс атакует случайного живого героя
                    target = random.choice([h for h in self.party if h.is_alive])
                    self.boss.basic_attack(target)
                    logger.write(f"{self.boss.name} ходит.")

                # Применяем эффекты
                for effect, target in self.effects[:]:
                    effect.apply(target)
                    if effect.duration <= 0:
                        self.effects.remove((effect, target))

            # -------------------- Конец боя --------------------
            if self.boss.is_alive:
                logger.write("❌ Пати проиграла! Босс победил!")
            else:
                logger.write("✅ Босс повержен!")


In [28]:




# ==================== Блок 4. Запуск игры ====================
if __name__ == "__main__":
    print("Добро пожаловать в 'Пати против Босса'! 🎮")

    # Ввод имён персонажей
    tank_name = input("Введите имя танка: ")
    warrior_name = input("Введите имя воина: ")
    mage_name = input("Введите имя мага: ")
    healer_name = input("Введите имя лекаря: ")
    assassin_name = input("Введите имя ассасина: ")

    # Выбор сложности
    print("\nВыберите сложность игры:")
    print("1 - Лёгкая")
    print("2 - Средняя")
    print("3 - Тяжёлая")
    choice = input("Ваш выбор: ")
    if choice == "1":
        difficulty = 0.7
    elif choice == "3":
        difficulty = 1.3
    else:
        difficulty = 1.0

    # Создаём героев
    party = [
        Tank(tank_name, hp=120, mp=30, strength=15, agility=5, intellect=5, defense=10),
        Warrior(warrior_name, hp=100, mp=30, strength=20, agility=7, intellect=5),
        Mage(mage_name, hp=80, mp=100, strength=5, agility=7, intellect=25),
        Healer(healer_name, hp=90, mp=100, strength=5, agility=6, intellect=20),
        Assassin(assassin_name, hp=70, mp=30, strength=20, agility=12, intellect=5, crit_chance=0.3)
    ]

    # Создаём босса
    class Boss(Character):
        def __init__(self, name, hp, mp, strength, agility, intellect, level=1):
            super().__init__(name, level, hp, mp, strength, agility, intellect)

        def basic_attack(self, target):
            damage = self.strength
            target.hp -= damage
            print(f"{self.name} наносит {damage} урона {target.name}!")

        def use_skill(self, party):
            print(f"{self.name} использует мощный удар по всем!")
            for hero in party:
                if hero.is_alive:
                    hero.hp -= self.strength
                    print(f"{hero.name} получает {self.strength} урона!")

    boss = Boss(
        "Дракон",
        hp=int(300 * difficulty),
        mp=int(50 * difficulty),
        strength=int(20 * difficulty),
        agility=int(8 * difficulty),
        intellect=int(10 * difficulty)
    )

    # Запуск боя
    game = Battle(party, boss)
    game.run()

Добро пожаловать в 'Пати против Босса'! 🎮
Введите имя танка: сава
Введите имя воина: петя
Введите имя мага: егор
Введите имя лекаря: даня
Введите имя ассасина: вова

Выберите сложность игры:
1 - Лёгкая
2 - Средняя
3 - Тяжёлая
Ваш выбор: 1
⚔️ Начало боя!

Ход Пати:
1. сава (Lvl 1) HP:120 MP:30
2. петя (Lvl 1) HP:100 MP:30
3. егор (Lvl 1) HP:80 MP:100
4. даня (Lvl 1) HP:90 MP:100
5. вова (Lvl 1) HP:70 MP:30
Выберите героя (1-5): 1

Выберите действие для сава:
1 - Basic Attack
2 - Use Skill
Ваш выбор: 1
сава бьёт Дракон на 15 урона!
сава сделал ход.

Ход босса Дракон:
Дракон наносит 14 урона петя!
Дракон ходит.

Ход Пати:
1. сава (Lvl 1) HP:120 MP:30
2. петя (Lvl 1) HP:86 MP:30
3. егор (Lvl 1) HP:80 MP:100
4. даня (Lvl 1) HP:90 MP:100
5. вова (Lvl 1) HP:70 MP:30
Выберите героя (1-5): 1

Выберите действие для сава:
1 - Basic Attack
2 - Use Skill
Ваш выбор: 1
сава бьёт Дракон на 15 урона!
сава сделал ход.

Ход босса Дракон:
Дракон наносит 14 урона петя!
Дракон ходит.

Ход Пати:
1. сава (Lvl